In [26]:
# run only once
!pip install duckduckgo_search

## How to search for images

In [2]:
from duckduckgo_search import DDGS

def search_images(keyword, max_results=10):
    with DDGS() as ddgs:
        images = ddgs.images(
            keyword,
            max_results=max_results
        )
        return [img['image'] for img in images]

**Example:**

Note we got less images than we asked for.

In [7]:
keyword = "teddybear"
image_urls = search_images(keyword, 1000)
len(image_urls)

418

Let us take a random image and look at it:

In [8]:
image_urls[123]

'https://i.etsystatic.com/23408950/r/il/4ed2a5/3977829066/il_fullxfull.3977829066_f608.jpg'

![](https://i.etsystatic.com/23408950/r/il/4ed2a5/3977829066/il_fullxfull.3977829066_f608.jpg)

## How to download images

In [18]:
import os
import requests
from urllib.parse import urlparse
import warnings

def download_image(url, folder, custom_name=None, verbose=True):
    # Create the folder if it doesn't exist
    os.makedirs(folder, exist_ok=True)

    # Get the filename from the URL or use the custom name
    if custom_name:
        filename = custom_name
    else:
        filename = os.path.basename(urlparse(url).path)
        if not filename:
            filename = 'image.jpg'  # Default filename if none is found in the URL

    # Ensure the filename has an extension
    if not os.path.splitext(filename)[1]:
        filename += '.jpg'

    filepath = os.path.join(folder, filename)

    # If the file already exists, append a number to make it unique
    base, extension = os.path.splitext(filepath)
    counter = 1
    while os.path.exists(filepath):
        filepath = f"{base}_{counter}{extension}"
        counter += 1

    try:
        # Send a GET request to the URL with a timeout of 10 seconds
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Check if the content type is an image
        content_type = response.headers.get('content-type', '')
        if not content_type.startswith('image'):
            if verbose:
                warnings.warn(f"The URL does not point to an image. Content-Type: {content_type}")
            return False

        # Write the image content to the file
        with open(filepath, 'wb') as f:
            f.write(response.content)

        if verbose:
            print(f"Image successfully downloaded: {filepath}")
        return True

    except requests.exceptions.Timeout:
        if verbose: 
            warnings.warn(f"Download timed out for URL: {url}")
    except requests.exceptions.HTTPError as e:
        if verbose: 
            warnings.warn(f"HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        if verbose: 
            warnings.warn(f"An error occurred while downloading the image: {e}")
    except IOError as e:
        if verbose: 
            warnings.warn(f"An error occurred while writing the file: {e}")

    return False

Let us donwload all teddybears into separate folder.

In [20]:
from tqdm.notebook import tqdm

for i, url in enumerate(tqdm(image_urls)):
    download_image(url, "./dataset/teddybear/", f'image{i}.jpg', verbose=False)

  0%|          | 0/418 [00:00<?, ?it/s]

## How to resize all images to 28x28

In [ ]:
# ...
from torchvision import datasets, transforms
# ...

# Data transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
])

# Load the dataset
dataset = datasets.ImageFolder(root='./dataset', transform=transform)

Do not forget the images are RGB, not greyscale as FashionMNIST. Therefore, they will have **three** channels on input, not one.

In [ ]:
# Define the simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # Input channels = 3 (RGB), output channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        # ...